In [22]:
import numpy as np

#? Note: from top left

def draw(l):
    size = len(l)
    s = ""
    s+="-"*(4*size+1)
    s+="\n"
    for i in range(size):
        s+="|"
        for j in range(size):
            s+=" "
            s+=str(l[i][j])
            s+=" |"
        s+="\n"
        s+="-"*(4*size+1)
        s+="\n"
    return s

class slot:
    def __init__(self, piece='d'):
        self.piece = piece  

    def change_piece(self, new_piece):
        self.piece = new_piece

def generate_board(board_size):
    board = []
    for i in range(board_size):
        board.append([slot() for i in range(board_size)])
    return board

def transform(array):
    l = []
    for i in array:
        temp = []
        for j in i:
            temp.append(slot(j))
        l.append(temp)
    return np.array(l)

class board:
    def __init__(self, auto=True, array=np.array([]), size=4):
        if not auto:
            self.board = transform(array)
        else:
            self.board = generate_board(size)
        self.size=size

    def __str__(self):
        view = []
        for row in self.board:
            temp = []
            for obj in row:
                temp.append(obj.piece)
            view.append(temp)
        return draw(view)
    
    def set_piece(self, row, col, piece):
        self.board[row][col].change_piece(piece)

    def place_piece(self, height, width, row, col, piece):
        for y in range(height):
            for x in range(width):
                self.board.set_piece(row+y,col+x,piece)

        

In [23]:
main = board()

In [24]:
print(main)

-----------------
| d | d | d | d |
-----------------
| d | d | d | d |
-----------------
| d | d | d | d |
-----------------
| d | d | d | d |
-----------------



In [25]:
main.set_piece(0,1,'g')

In [26]:
test = board(auto=False, array=np.array([[1,1,2,3],[1,2,8,4],[1,5,8,6],[0,7,2,4]]))

In [27]:
print(test)

-----------------
| 1 | 1 | 2 | 3 |
-----------------
| 1 | 2 | 8 | 4 |
-----------------
| 1 | 5 | 8 | 6 |
-----------------
| 0 | 7 | 2 | 4 |
-----------------

